In [11]:
from argparse import Namespace

import torch
import torch.nn as nn
import torch.nn.functional as F
from logger import init_logger as Init_Logger

# from pl_bolts.optimizers.lr_scheduler import LinearWarmupCosineAnnealingLR
from lr_scheduler import LinearWarmupCosineAnnealingLR
from pytorch_lightning import LightningModule
from torch.optim.lr_scheduler import MultiStepLR, ReduceLROnPlateau
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2PhonemeCTCTokenizer,
    Wav2Vec2Processor,
    PretrainedConfig,
Wav2Vec2Config
)

In [12]:
config_path = "/home/ay/.cache/huggingface/hub/models--facebook--wav2vec2-base-960h/snapshots/22aad52d435eb6dbaf354bdad9b0da84ce7d6156/config.json"
config = Wav2Vec2Config.from_json_file(config_path)

In [24]:
model = Wav2Vec2ForCTC(
    config=config
)

In [30]:
model2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/home/ay/anaconda3/envs/torch/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2

In [25]:
model.wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0

Parameter containing:
tensor([[[1.4158, 1.4146, 1.4132, 1.4148, 1.4172, 1.4123, 1.4149, 1.4162,
          1.4189, 1.4182, 1.4210, 1.4173, 1.4133, 1.4147, 1.4122, 1.4104,
          1.4098, 1.4114, 1.4143, 1.4139, 1.4112, 1.4164, 1.4164, 1.4164,
          1.4106, 1.4150, 1.4164, 1.4126, 1.4118, 1.4148, 1.4141, 1.4174,
          1.4185, 1.4150, 1.4172, 1.4170, 1.4100, 1.4178, 1.4114, 1.4136,
          1.4162, 1.4135, 1.4132, 1.4152, 1.4089, 1.4059, 1.4172, 1.4114,
          1.4106, 1.4136, 1.4194, 1.4057, 1.4215, 1.4137, 1.4176, 1.4183,
          1.4145, 1.4203, 1.4204, 1.4172, 1.4135, 1.4088, 1.4085, 1.4174,
          1.4138, 1.4137, 1.4168, 1.4121, 1.4153, 1.4202, 1.4157, 1.4210,
          1.4109, 1.4141, 1.4203, 1.4123, 1.4161, 1.4125, 1.4118, 1.4152,
          1.4228, 1.4131, 1.4130, 1.4143, 1.4121, 1.4100, 1.4143, 1.4162,
          1.4123, 1.4170, 1.4126, 1.4159, 1.4129, 1.4162, 1.4231, 1.4152,
          1.4133, 1.4173, 1.4146, 1.4137, 1.4161, 1.4093, 1.4187, 1.4232,
          1.4160

In [31]:
print(model2.wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0,
model2.wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1,
model2.wav2vec2.masked_spec_embed)


Parameter containing:
tensor([[[0.4319, 0.1718, 0.1194, 0.1323, 0.1245, 0.1046, 0.1022, 0.1040,
          0.1171, 0.1301, 0.1311, 0.1234, 0.1054, 0.1218, 0.1247, 0.1161,
          0.1128, 0.1166, 0.1236, 0.1333, 0.1237, 0.1166, 0.1210, 0.1292,
          0.1333, 0.1212, 0.1309, 0.1311, 0.1373, 0.1384, 0.1432, 0.1343,
          0.1472, 0.1759, 0.1549, 0.1453, 0.1960, 0.2131, 0.1648, 0.1699,
          0.1526, 0.1703, 0.1725, 0.1912, 0.1981, 0.1995, 0.2181, 0.2292,
          0.2542, 0.2854, 0.3005, 0.3582, 0.4033, 0.4707, 0.5591, 0.6724,
          0.7935, 0.9917, 1.2051, 1.4795, 1.8555, 2.5332, 4.4180, 9.4062,
          8.6328, 9.0312, 4.3516, 2.4961, 1.8057, 1.4248, 1.1201, 0.9199,
          0.7422, 0.6431, 0.5562, 0.4788, 0.4209, 0.3691, 0.3354, 0.3071,
          0.2751, 0.2559, 0.2316, 0.2281, 0.1998, 0.2058, 0.1880, 0.1847,
          0.1880, 0.2241, 0.1669, 0.1617, 0.1528, 0.1561, 0.1599, 0.1409,
          0.1614, 0.1329, 0.1464, 0.1509, 0.1284, 0.1379, 0.1245, 0.1271,
          0.1278

In [3]:
from transformers import Wav2Vec2ForCTC, WavLMForCTC


class BaseModel(nn.Module):
    """
    BaseFeaturesExtractor class that will extract features according to the type of model
    https://huggingface.co/blog/fine-tune-wav2vec2-english
    """

    def __init__(self, params):
        super().__init__()
        self.params = params

    def forward(self, x):
        outputs = self.model(x)
        return outputs


class Wav2Vec2(BaseModel):
    """
    https://huggingface.co/docs/transformers/v4.16.2/en/model_doc/wav2vec2#transformers.Wav2Vec2ForCTC
    """

    def __init__(self, params):
        super().__init__(params)

        self.model = Wav2Vec2ForCTC.from_pretrained(params.pretrained_name)
        in_features = self.model.lm_head.in_features
        self.model.lm_head = nn.Linear(in_features=in_features, out_features=self.params.vocab_size)


class WavLM(BaseModel):
    """
    https://huggingface.co/docs/transformers/model_doc/wavlm#transformers.WavLMForCTC
    """

    def __init__(self, params):
        super().__init__(params)
        self.model = WavLMForCTC.from_pretrained(params.pretrained_name)
        in_features = self.model.lm_head.in_features
        self.model.lm_head = nn.Linear(in_features=in_features, out_features=self.params.vocab_size)
        print(self.model.lm_head.weight.shape)


network_param = Namespace(
    network_name="WavLM",
    # pretrained_name="microsoft/wavlm-base",
    pretrained_name="facebook/wav2vec2-base-960h",
    freeze=True,
    freeze_transformer=False,
    eos_token="</s>",
    bos_token="<s>",
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token="|",
    vocab_size=200,
)